In [3]:
!pip install jupyterlab-nvidia-nsight

In [1]:
import numpy as np
from numba import cuda, int64
import time

TAILLE_BIGNUM = 64  
# taille de clé RSA de 2048 bits (64 * élément de 32 bits)

# Multiplication BIGNUM sur CPU
def multiplication_bignum_cpu(a, b):
    resultat = np.zeros(TAILLE_BIGNUM * 2, dtype=np.uint64)
    for i in range(TAILLE_BIGNUM):
        retenue = 0
        for j in range(TAILLE_BIGNUM):
            if i + j < len(resultat):
                temp = resultat[i + j] + int(a[i]) * int(b[j]) + retenue
                resultat[i + j] = temp & 0xFFFFFFFF
                retenue = temp >> 32
    return resultat[:TAILLE_BIGNUM]

# Génère des p et q FIXES pour test réaliste
def generer_cle_rsa_bignum():
    p = np.array([0] * TAILLE_BIGNUM, dtype=np.uint32)
    q = np.array([0] * TAILLE_BIGNUM, dtype=np.uint32)
    p[0] = 49993
    q[0] = 50021
    n = multiplication_bignum_cpu(p, q)
    return p, q, n

# Modulo BIGNUM
def bignum_mod_uint(bignum, diviseur):
    reste = 0
    for i in reversed(range(TAILLE_BIGNUM)):
        reste = (reste << 32) + int(bignum[i])
        reste = reste % diviseur
    return reste

# CPU Bruteforce
def factorisation_bruteforce_cpu_bignum(n_bignum):
    for p in range(3, 2**16, 2):
        if bignum_mod_uint(n_bignum, p) == 0:
            return p
    return None

# GPU Kernel optimisé
@cuda.jit
def kernel_factorisation_bignum_gpu(n, facteurs, compteur):
    shared_n = cuda.shared.array(shape=TAILLE_BIGNUM, dtype=int64)
    idx = cuda.grid(1)
    thread_id = cuda.threadIdx.x
    if thread_id < TAILLE_BIGNUM:
        shared_n[thread_id] = n[thread_id]
    cuda.syncthreads()

    if idx < 2**16:
        candidat = idx * 2 + 3
        reste = 0
        for i in range(TAILLE_BIGNUM - 1, -1, -1):
            reste = (reste << 32) + shared_n[i]
            reste = reste % candidat
        if reste == 0:
            index = cuda.atomic.add(compteur, 0, 1)
            facteurs[index, 0] = candidat

# GPU Launcher
def factoriser_bignum_gpu(n_bignum):
    max_facteurs = 10
    facteurs = cuda.pinned_array((max_facteurs, 2), dtype=np.int64)
    compteur = cuda.pinned_array(1, dtype=np.int32)
    facteurs[:] = 0
    compteur[:] = 0

    d_n = cuda.to_device(n_bignum)
    d_facteurs = cuda.to_device(facteurs)
    d_compteur = cuda.to_device(compteur)

    threads_par_bloc = 256
    blocs_par_grille = ((2**16) + threads_par_bloc - 1) // threads_par_bloc

    # Mesure du temps avec time.time()
    t1 = time.time()
    kernel_factorisation_bignum_gpu[blocs_par_grille, threads_par_bloc](d_n, d_facteurs, d_compteur)
    cuda.synchronize()
    t2 = time.time()

    d_facteurs.copy_to_host(facteurs)
    print(f"Temps GPU (Kernel seul) : {(t2 - t1) * 1000:.3f} ms")
    return facteurs

if __name__ == "__main__":
    print("=== Génération des clés RSA BIGNUM ===")
    debut = time.time()
    p, q, n_bignum = generer_cle_rsa_bignum()
    fin = time.time()
    print(f"Génération terminée en {(fin - debut) * 1000:.3f} ms")

    print("\n=== Factorisation CPU ===")
    debut_cpu = time.time()
    facteur_cpu = factorisation_bruteforce_cpu_bignum(n_bignum)
    fin_cpu = time.time()
    cpu_time_ms = (fin_cpu - debut_cpu) * 1000
    print(f"Temps CPU : {cpu_time_ms:.3f} ms")
    print(f"Facteur trouvé par CPU : p = {facteur_cpu}")

    print("\n=== Factorisation GPU optimisée ===")
    debut_gpu = time.time()
    facteurs = factoriser_bignum_gpu(n_bignum)
    fin_gpu = time.time()
    gpu_time_ms = (fin_gpu - debut_gpu) * 1000
    print(f"Temps GPU (Total incluant transfert mémoire) : {gpu_time_ms:.3f} ms")
    print("Facteurs GPU trouvés :")
    print(facteurs)

    # Vérification si le GPU trouve le même facteur
    correspondance = any(facteur_cpu == f[0] for f in facteurs if f[0] != 0)
    if correspondance:
        print(f"\n CORRESPONDANCE GPU/CPU : facteur = {facteur_cpu}")
    else:
        print("\n Pas de correspondance GPU/CPU")

    # Calcul de l'accélération
    speedup = cpu_time_ms / gpu_time_ms
    print(f"\n Accélération GPU vs CPU : {speedup:.2f}x")


=== Génération des clés RSA BIGNUM ===
Génération terminée en 2.662 ms

=== Factorisation CPU ===
Temps CPU : 209.740 ms
Facteur trouvé par CPU : p = 49993

=== Factorisation GPU optimisée ===
Temps GPU (Kernel seul) : 259.585 ms
Temps GPU (Total incluant transfert mémoire) : 783.572 ms
Facteurs GPU trouvés :
[[49993     0]
 [50021     0]
 [    0     0]
 [    0     0]
 [    0     0]
 [    0     0]
 [    0     0]
 [    0     0]
 [    0     0]
 [    0     0]]

 CORRESPONDANCE GPU/CPU : facteur = 49993

 Accélération GPU vs CPU : 0.27x


Generating '/tmp/nsys-report-1bd9.qdstrm'
[1/2] [========================100%] RTC:Untitled1_2025-03-23T11:25:02.nsys-rep
[2/2] [========================100%] RTC:Untitled1_2025-03-23T11:25:02.sqlite
Generated:
	/home/jovyan/RTC:Untitled1_2025-03-23T11:25:02.nsys-rep
	/home/jovyan/RTC:Untitled1_2025-03-23T11:25:02.sqlite
Generating SQLite file /home/jovyan/RTC:Untitled1_2025-03-23T11:25:02.sqlite from /home/jovyan/RTC:Untitled1_2025-03-23T11:25:02.nsys-rep
Processing [/home/jovyan/RTC:Untitled1_2025-03-23T11:25:02.sqlite] with [/opt/nvidia/nsight-systems/2025.2.1/host-linux-x64/reports/nvtx_sum.py]... 

 ** NVTX Range Summary (nvtx_sum):

Report was successfully run, but no data was returned.

Processing [/home/jovyan/RTC:Untitled1_2025-03-23T11:25:02.sqlite] with [/opt/nvidia/nsight-systems/2025.2.1/host-linux-x64/reports/osrt_sum.py]... 

 ** OS Runtime Summary (osrt_sum):

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)     Med (ns)    Min (ns)   Max (ns)   StdDev (ns)           Na